In [1]:
from scipy.io import wavfile
import numpy as np
import torch

# Read WAV file
sample_rate, audio_data = wavfile.read("C:\\Users\\khwez\\Desktop\\Drake Best I Ever Had.wav")

# Convert stereo to mono by averaging the two channels
if len(audio_data.shape) == 2:
    audio_data = np.mean(audio_data, axis=1).astype(audio_data.dtype)

# Define segment length
segment_length = sample_rate  # 1 second

# Calculate total segments and the length of the last segment
total_segments = len(audio_data) // segment_length
remainder = len(audio_data) % segment_length

# Initialize a zero-padded array
padded_audio_data = np.zeros((total_segments + 1) * segment_length, dtype=audio_data.dtype)
padded_audio_data[:len(audio_data)] = audio_data

# Create segments
segments = np.array([padded_audio_data[i:i + segment_length] for i in range(0, len(padded_audio_data), segment_length)])

# Convert to PyTorch tensor
tensor_segments = torch.IntTensor(segments)

# Now tensor_segments is ready to be used in your PyTorch model


In [2]:
pip install librosa


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\khwez\\AppData\\Roaming\\Python\\Python39\\site-packages\\~umpy.libs\\libopenblas64__v0.3.23-293-gc2f4bdbb-gcc_10_3_0-65e29aac85b9409a6008e2dc84b1cc09.dll'
Consider using the `--user` option or check the permissions.




  Obtaining dependency information for librosa from https://files.pythonhosted.org/packages/e2/a2/4f639c1168d7aada749a896afb4892a831e2041bebdcf636aebfe9e86556/librosa-0.10.1-py3-none-any.whl.metadata
     ---------------------------------------- 0.0/377.0 kB ? eta -:--:--
     --- --------------------------------- 30.7/377.0 kB 660.6 kB/s eta 0:00:01
     ---- -------------------------------- 41.0/377.0 kB 653.6 kB/s eta 0:00:01
     -------------------- ----------------- 204.8/377.0 kB 1.6 MB/s eta 0:00:01
     -------------------------------------- 377.0/377.0 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Obtaining dependency information for scikit-learn>=0.20.0 from https://files.pythonhosted.org/packages/1b/7c/66d34dc231a309cc3a7f3eeb848319be3b616fd2918db302aaef727eff4a/scikit_learn-1.3.1-cp39-cp39-win_amd64.whl.metadata
  Obtaining dependency information for joblib>=0.14 from https://files.pythonhos

In [3]:
pip install librosa --user


  Obtaining dependency information for librosa from https://files.pythonhosted.org/packages/e2/a2/4f639c1168d7aada749a896afb4892a831e2041bebdcf636aebfe9e86556/librosa-0.10.1-py3-none-any.whl.metadata
  Using cached librosa-0.10.1-py3-none-any.whl.metadata (8.3 kB)
  Using cached audioread-3.0.0-py3-none-any.whl
  Obtaining dependency information for scikit-learn>=0.20.0 from https://files.pythonhosted.org/packages/1b/7c/66d34dc231a309cc3a7f3eeb848319be3b616fd2918db302aaef727eff4a/scikit_learn-1.3.1-cp39-cp39-win_amd64.whl.metadata
  Using cached scikit_learn-1.3.1-cp39-cp39-win_amd64.whl.metadata (11 kB)
  Obtaining dependency information for joblib>=0.14 from https://files.pythonhosted.org/packages/10/40/d551139c85db202f1f384ba8bcf96aca2f329440a844f924c8a0040b6d02/joblib-1.3.2-py3-none-any.whl.metadata
  Using cached joblib-1.3.2-py3-none-any.whl.metadata (5.4 kB)
  Obtaining dependency information for numba>=0.51.0 from https://files.pythonhosted.org/packages/2c/31/121c11175e49482ef5

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [7]:
from scipy.io import wavfile
import numpy as np
import torch
import librosa
from sklearn.preprocessing import StandardScaler

# Read WAV file
sample_rate, audio_data = wavfile.read("C:\\Users\\khwez\\Desktop\\Drake Best I Ever Had.wav")

# Convert stereo to mono
if len(audio_data.shape) == 2:
    audio_data = np.mean(audio_data, axis=1).astype(audio_data.dtype)

# Ensure audio_data is float32 for librosa
audio_data = audio_data.astype(np.float32)

# Define segment length
segment_length = sample_rate  # 1 second

# Calculate total segments and pad if necessary
total_segments = len(audio_data) // segment_length
remainder = len(audio_data) % segment_length

# Pad audio data with zeros to make it perfectly divisible into segments
if remainder != 0:
    audio_data = np.pad(audio_data, (0, segment_length - remainder), 'constant', constant_values=0)

# Segment audio data
segments = np.array([audio_data[i:i + segment_length] for i in range(0, len(audio_data), segment_length)])

# Initialize feature arrays
mfccs = []
chroma = []
contrast = []

# Extract features for each segment
for segment in segments:
    mfcc = librosa.feature.mfcc(y=segment, sr=sample_rate, n_mfcc=13)
    chroma_stft = librosa.feature.chroma_stft(y=segment, sr=sample_rate)
    spectral_contrast = librosa.feature.spectral_contrast(y=segment, sr=sample_rate)

    mfccs.append(mfcc)
    chroma.append(chroma_stft)
    contrast.append(spectral_contrast)

# Convert list of numpy arrays to a single numpy array
mfccs_np = np.array(mfccs)
chroma_np = np.array(chroma)
contrast_np = np.array(contrast)

# Flatten the array for standardization
mfccs_reshaped = mfccs_np.reshape(-1, mfccs_np.shape[-1])

# Standardize
scaler = StandardScaler()
mfccs_scaled = scaler.fit_transform(mfccs_reshaped)

# Reshape back to original shape
mfccs_scaled = mfccs_scaled.reshape(mfccs_np.shape)

# Convert to PyTorch tensors
tensor_mfccs = torch.FloatTensor(mfccs_scaled)
tensor_chroma = torch.FloatTensor(chroma_np)
tensor_contrast = torch.FloatTensor(contrast_np)

# Save tensors
torch.save(tensor_mfccs, 'tensor_mfccs.pt')


C:\Users\khwez\AppData\Roaming\Python\Python39\site-packages\librosa\core\pitch.py:101: UserWarning: Trying to estimate tuning from empty frequency set.
  return pitch_tuning(


In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from scipy.io import wavfile
import numpy as np
import librosa
from sklearn.preprocessing import StandardScaler

# Read WAV file
sample_rate, audio_data = wavfile.read("C:\\Users\\khwez\\Desktop\\Drake Best I Ever Had.wav")

# Convert stereo to mono
if len(audio_data.shape) == 2:
    audio_data = np.mean(audio_data, axis=1).astype(audio_data.dtype)

# Ensure audio_data is float32 for librosa
audio_data = audio_data.astype(np.float32)

# Define segment length
segment_length = sample_rate  # 1 second

# Calculate total segments and pad if necessary
total_segments = len(audio_data) // segment_length
remainder = len(audio_data) % segment_length

# Pad audio data with zeros to make it perfectly divisible into segments
if remainder != 0:
    audio_data = np.pad(audio_data, (0, segment_length - remainder), 'constant', constant_values=0)

# Segment audio data
segments = np.array([audio_data[i:i + segment_length] for i in range(0, len(audio_data), segment_length)])

# Initialize feature arrays
mfccs = []
chroma = []
contrast = []

# Extract features for each segment
for segment in segments:
    mfcc = librosa.feature.mfcc(y=segment, sr=sample_rate, n_mfcc=13)
    chroma_stft = librosa.feature.chroma_stft(y=segment, sr=sample_rate)
    spectral_contrast = librosa.feature.spectral_contrast(y=segment, sr=sample_rate)

    mfccs.append(mfcc)
    chroma.append(chroma_stft)
    contrast.append(spectral_contrast)

# Convert list of numpy arrays to a single numpy array
mfccs_np = np.array(mfccs)
chroma_np = np.array(chroma)
contrast_np = np.array(contrast)

# Flatten the array for standardization
mfccs_reshaped = mfccs_np.reshape(-1, mfccs_np.shape[-1])

# Standardize
scaler = StandardScaler()
mfccs_scaled = scaler.fit_transform(mfccs_reshaped)

# Reshape back to original shape
mfccs_scaled = mfccs_scaled.reshape(mfccs_np.shape)

# Convert to PyTorch tensors
tensor_mfccs = torch.FloatTensor(mfccs_scaled)
tensor_chroma = torch.FloatTensor(chroma_np)
tensor_contrast = torch.FloatTensor(contrast_np)

# Save tensors
torch.save(tensor_mfccs, 'tensor_mfccs.pt')

# Model Architecture
class AudioClassifier(nn.Module):
    def __init__(self):
        super(AudioClassifier, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 3)
        self.conv2 = nn.Conv2d(16, 32, 3)
        self.conv3 = nn.Conv2d(32, 64, 3)
        self.fc1 = nn.Linear(64 * 6 * 6, 128)  # Adjust dimensions based on your MFCC shape
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)  # 10 classes, adjust as per your use-case

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), 2)
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = F.max_pool2d(F.relu(self.conv3(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return F.log_softmax(x, dim=1)

    def num_flat_features(self, x):
        size = x.size()[1:]  # All dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

# Instantiate the model
model = AudioClassifier()


C:\Users\khwez\AppData\Roaming\Python\Python39\site-packages\librosa\core\pitch.py:101: UserWarning: Trying to estimate tuning from empty frequency set.
  return pitch_tuning(


In [9]:
from pydub import AudioSegment
import librosa
import numpy as np

# Load an audio file
audio = AudioSegment.from_file("C:\\Users\\khwez\\Desktop\\Drake Best I Ever Had.wav")

# Resample to 16 kHz
audio = audio.set_frame_rate(16000)

# Define a function to extract features from an audio segment
def extract_features(audio_segment, sample_rate):
    # Convert the audio segment to a numpy array of floating-point values
    audio_data = np.array(audio_segment.get_array_of_samples(), dtype=np.float32)

    # Extract MFCCs (Mel-frequency cepstral coefficients)
    mfccs = librosa.feature.mfcc(y=audio_data, sr=sample_rate, n_mfcc=13)

    # Extract pitch
    pitch, _ = librosa.piptrack(y=audio_data, sr=sample_rate)

    return mfccs, pitch

# Sample usage
sample_rate = 16000  # Adjust to match your audio's sample rate
mfccs, pitch = extract_features(audio, sample_rate)


In [7]:
import librosa
import numpy as np

# Define a function to extract features from an audio segment
def extract_features(audio_segment, sample_rate):
    # Convert the audio segment to a numpy array
    audio_data = np.array(audio_segment.get_array_of_samples())

    # Extract MFCCs (Mel-frequency cepstral coefficients)
    mfccs = librosa.feature.mfcc(y=audio_data, sr=sample_rate, n_mfcc=13)

    # Extract pitch
    pitch, _ = librosa.piptrack(y=audio_data, sr=sample_rate)

    return mfccs, pitch

# Sample usage
sample_rate = 16000  # Adjust to match your audio's sample rate

# Replace 'audio_segment' with the actual audio segment you want to process
mfccs, pitch = extract_features(your_audio_segment, sample_rate)


NameError: name 'your_audio_segment' is not defined

In [2]:
pip install pydub


Note: you may need to restart the kernel to use updated packages.


In [8]:
from torch.utils.data import random_split

# Determine dataset size
n_samples = tensor_mfccs.shape[0]

# Define ratios
train_size = int(0.7 * n_samples)
val_size = int(0.15 * n_samples)
test_size = n_samples - train_size - val_size

# Split the dataset
train_set, val_set, test_set = random_split(tensor_mfccs, [train_size, val_size, test_size])


In [2]:
pip install porch 

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement porch (from versions: none)
ERROR: No matching distribution found for porch


In [3]:
pip install torch 

     ---------------------------------------- 0.0/172.4 MB ? eta -:--:--
     -------------------------------------- 0.0/172.4 MB 330.3 kB/s eta 0:08:42
     -------------------------------------- 0.1/172.4 MB 787.7 kB/s eta 0:03:39
     ---------------------------------------- 0.2/172.4 MB 1.3 MB/s eta 0:02:12
     ---------------------------------------- 0.3/172.4 MB 2.1 MB/s eta 0:01:22
     ---------------------------------------- 0.8/172.4 MB 3.7 MB/s eta 0:00:47
     ---------------------------------------- 1.7/172.4 MB 6.3 MB/s eta 0:00:28
      --------------------------------------- 2.2/172.4 MB 6.9 MB/s eta 0:00:25
      --------------------------------------- 3.1/172.4 MB 8.6 MB/s eta 0:00:20
      -------------------------------------- 4.3/172.4 MB 10.6 MB/s eta 0:00:16
     - ------------------------------------- 6.1/172.4 MB 13.4 MB/s eta 0:00:13
     - ------------------------------------- 7.5/172.4 MB 15.1 MB/s eta 0:00:11
     - ------------------------------------- 8.